<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [0]:
import requests

data = []

r = requests.get('https://www.gutenberg.org/files/100/100-0.txt')

In [0]:
r.encoding = 'utf-8'

data = r.text.split('\n')

In [0]:
text = "".join(set(data))

# Unique Characters
chars = []

for i in text:
    if i not in chars:
        chars.append(i)

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [5]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1068142


In [0]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [7]:
x.shape

(1068142, 40, 107)

In [8]:
y.shape

(1068142, 107)

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
shakespeare = []

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        shakespeare.append(sentence)

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [13]:
history = model.fit(x, y,
          batch_size=256,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
4167/4173 [============================>.] - ETA: 0s - loss: 2.3559
----- Generating text after Epoch: 0
    But"
  CEUCETE. He tleen’s prei
4173/4173 [==============================] - 39s 9ms/step - loss: 2.3555
Epoch 2/20
4166/4173 [============================>.] - ETA: 0s - loss: 2.0020
----- Generating text after Epoch: 1
    That died in honour "
Thee; a gainit there I wim, 
4173/4173 [==============================] - 39s 9ms/step - loss: 2.0020
Epoch 3/20
4168/4173 [============================>.] - ETA: 0s - loss: 1.8908
----- Generating text after Epoch: 2
say he com"
Yet the princersing
4173/4173 [==============================] - 40s 9ms/step - loss: 1.8907
Epoch 4/20
4173/4173 [==============================] - ETA: 0s - loss: 1.8075
----- Generating text after Epoch: 3
O heat, dry up my brains. Tears seve"
Masted be hearts lound preasien a mosted lovilys, Britured not a, incl
4173/4173 [==============================] - 39s 9ms/step - loss: 1.8075
Epoch 5/20
4

In [363]:
from pprint import pprint
pprint(''.join(set(shakespeare)).replace('\r', ' ')[:])

TypeError: ignored

In [0]:
text = "".join(set(data))
text = text.replace('\r', ' ').split(' ')
text = text[:50000]
words = set(text)

In [0]:
# Lookup Tables
word_int = {w:i for i, w in enumerate(words)} 
int_word = {i:w for i, w in enumerate(words)}

maxlen = 5
step = 3

encoded = [word_int[w] for w in text]

In [25]:
sequences = []
next_word = []

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_word.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  16665


In [26]:
print((len(sequences), maxlen, len(words)))

(16665, 5, 10488)


In [0]:
x = np.zeros((len(sequences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sequences), len(words)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, word in enumerate(sequence):
        x[i,t,word] = 1
        
    y[i, next_word[i]] = 1

In [94]:
x.shape

(16665, 5, 10488)

In [234]:
len(text)

50000

In [0]:
from tensorflow.keras.preprocessing import sequence
x = sequence.pad_sequences(x, maxlen=maxlen)

In [343]:
word_int

{'': 0,
 'happy': 1,
 'Said': 2,
 'met.': 3,
 'infinite,': 4,
 'afternoon;': 5,
 'quick': 6,
 'shrinks:': 7,
 'country’s': 8,
 'temple': 9,
 'children?': 10,
 'nation': 11,
 'Behind': 12,
 "pull'd": 13,
 'bulk,': 14,
 'unto': 15,
 'hangings,': 16,
 'Forum.': 17,
 'busines': 18,
 'change': 19,
 'HASTINGS.': 20,
 'OLD': 21,
 'Crispin’s': 22,
 'EVANS': 23,
 'breeding.': 24,
 'book,': 25,
 'jars': 26,
 'tale': 27,
 "'Where's": 28,
 'Troilus.': 29,
 'commanded': 30,
 'virtues': 31,
 'men': 32,
 'BUCKINGHAM,': 33,
 '“God': 34,
 'Sink': 35,
 'Battaile': 36,
 'Of': 37,
 'gods!': 38,
 'bids': 39,
 'sick.': 40,
 'distraction.': 41,
 'Claudio,': 42,
 'anger.': 43,
 'towards': 44,
 'knocks?': 45,
 'envy': 46,
 'Oberon.': 47,
 'Project': 48,
 'you’re': 49,
 'Phoebus': 50,
 'Longaville': 51,
 'contact': 52,
 'appal': 53,
 'mine,': 54,
 'queas': 55,
 'him;': 56,
 'marriage;': 57,
 'Page,': 58,
 'accordingly,': 59,
 'Exit,': 60,
 'weighed': 61,
 'nine': 62,
 'sorrows': 63,
 'power': 64,
 'lords.': 65,

In [0]:
shakespeare = []

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = []
    
    sentence = text[start_index: start_index + maxlen]
    generated.append(sentence)
    
    print(f'----- Generating with seed: "{[word for word in sentence]}"')
    sys.stdout.write(' '.join([str(x) for x in generated]))
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(words)))
        for t, word in enumerate(sentence):
          x_pred[0, t, word_int[word]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_word = int_word[next_index]
        
        sentence = sentence[1:] + [next_word]
        shakespeare.append(sentence)

        
        sys.stdout.write(next_word)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [405]:
model = Sequential()

model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, 
               input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_97"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_92 (LSTM)               (None, 128)               5435904   
_________________________________________________________________
dense_95 (Dense)             (None, 10488)             1352952   
Total params: 6,788,856
Trainable params: 6,788,856
Non-trainable params: 0
_________________________________________________________________


In [406]:
history = model.fit(x, y,
                    batch_size=256,
                    epochs=1,
                    callbacks=[print_callback])

66/66 [==============================] - ETA: 0s - loss: 0.3839 - accuracy: 0.1275
----- Generating text after Epoch: 0
----- Generating with seed: "['just', 'of', 'the', 'same', 'That']"
['just', 'of', 'the', 'same', 'That']justice.give-acheek,distinguish,ne'erwants-blindeknaveblessesyokepains;eternalmutedead?perplexherbs,unboundcool,dangerous.againstbees,That’sspites!witchcraftmaintained,George!”King'sway.adjacentadderheirdevis’dknownlanguesGREYbraceletdecideItalianstationacquittancegrief.whit.lodgd,Hector.heavens!girl,'OCTAVIA.saidPISTOL[_Aside_]Tranio_.]Romeprincelypersonalhorse?hathThenSoldiers.shalsuffrers;harmony.love?Gloucester,husband.gateBuckingham-finehunt;turnsNever,come!neighbour._queIago,Players._]stranger,gapingpublictalk.fist,SettingAnneconstrueEmbarkMark'dmad,moiety?lim’d,fallssheriff,breedmyself,—seen;panderspuissanceblessingabsence,leftsteedOCTAVIA.hurriesesperancecoat;ascendsstraw!punyouth,sameknewporridge.threwshakesRightpocket?MAN.martialbastardtomorrowsceneson.te

In [434]:
pprint(' '.join(set([' '.join(x) for x in shakespeare]))[-2000:])

(" ill besiege practices! too; allay Apothecary. Mine Harry's air, time’s "
 'subjects Death. man—how Pentapolis, left steed OCTAVIA. hurries esperance '
 'proscriptions, Richmond, potentates? conclusion. die the same That justice. '
 'give-a of the same That justice. retreat perishing flood Alla waiting nature '
 "lends fly— one? seat BOYET. harper's soldier. so. Officers provoke STRANGER. "
 "dropsies, again: odes Harry's air, Bosworth Amen, barks; others. foreigners "
 "HENRY. unproper attendants seal scan sighs forth. occulted George!” King's "
 'way. adjacent adder scan sighs forth. occulted guess’d. pun youth, same knew '
 'porridge. limb, stoccata weakens Controlment returning myself,— seen; '
 'panders puissance blessing subjects Death. man—how Pentapolis, drove his, '
 'repent Majesty! agrees supper, threw shakes Right pocket? MAN. habit. Air, '
 "end prepost’rously. merit; Officers trick Thesius, knit.' so HENRY. unproper "
 'attendants done; sense, strike. over-tedious. othe

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN